In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import sys

print("Working Environment Information:")
print("-" * 30)
print(f"Python version: {sys.version.split()[0]}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"openpyxl version: {openpyxl.__version__}")

Working Environment Information:
------------------------------
Python version: 3.12.12
pandas version: 2.2.2
numpy version: 2.0.2
openpyxl version: 3.1.5


# Dataset

In [ ]:
file_path = 'A2.xlsx'
df = pd.read_excel(file_path)

df1 = df.copy()
# print(df.info())
print(df.head())

   Patient Delivery Date Day of Week Delivery Time               CS type  \
0        1    2013-04-01      Monday      01:21:00      Unplanned-urgent   
1        2    2013-04-01      Monday      09:59:00                   NaN   
2        3    2013-04-01      Monday      11:23:00  Planned-not as sched   
3        4    2013-04-01      Monday      12:33:00      Unplanned-urgent   
4        5    2013-04-01      Monday      16:37:00      Unplanned-urgent   

          OR Arrival Time       OR Departure Time  Total Patient OR time  
0 2013-04-01 00:46:35.000 2013-04-01 03:51:24.000                  184.0  
1 2013-04-01 09:26:19.843 2013-04-01 10:40:31.843                   74.0  
2 2013-04-01 10:36:50.623 2013-04-01 12:01:40.623                   84.0  
3 2013-04-01 11:47:01.307 2013-04-01 13:05:30.307                   78.0  
4 2013-04-01 16:20:29.167 2013-04-01 17:30:20.167                   69.0  


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
df.dtypes

,0
Patient,int64
Delivery Date,object
Day of Week,object
Delivery Time,object
CS type,object
OR Arrival Time,datetime64[ns]
OR Departure Time,datetime64[ns]
Total Patient OR time,float64


# 1. Data Cleaning

### 1.1 - CS type

- 1.1.1 - Removing Rows with Unknown “CS Type”

  - The “CS Type” column contained some unknown cases that were neither classified as Planned nor Unplanned. Since these rows did not help forecast the number of Planned or Unplanned C-sections in the future, they were removed from the dataset.

  - since we are unable to determine whether it is planned or not

- 1.1.2 - Standardize CS type categories by merging similar unplanned subtypes


In [ ]:
# 1.1.1 Remove NULL
# df[df["CS type"].isna()] # df["CS type"].isna().sum()
df = df.dropna(subset=["CS type"])

In [ ]:
# 1.1.2 classify CS type

# df["CS type"].value_counts()

# Standardize CS type categories by merging similar unplanned subtypes
# ("Unplanned-urgent", "Unplanned-crash", "Unplanned-") into a single category "Unplanned"
df.loc[:, "CS type"] = df["CS type"].replace({
    "Unplanned-urgent": "Unplanned",
    "Unplanned-crash": "Unplanned",
    "Unplanned-": "Unplanned"
})

df["CS type"].value_counts()


,count
CS type,
Unplanned,1048
Planned-as scheduled,897
Planned-not as sched,213


In [ ]:
# % of each CS type
df["CS type"].value_counts(normalize=True) * 100


,proportion
CS type,
Unplanned,48.563485
Planned-as scheduled,41.566265
Planned-not as sched,9.870250


### 1.2 - Total Patient OR time - Order Matters

- 1.2.1 - remove rows with all three information missing (OR Arrival & Departure Time & Total OR Time)

- 1.2.2 - Exclude Total Patient OR time <= 10 min & NOT NULL
  - fail to take place?
  - impossible to perform a surgery
  - which will underestimate
  - keep NULL value for following imputation

- 1.2.3 - Handelling Missing value
  - imputation - replaced using the median Total Patient OR Time of that specific CS Type.




In [ ]:
# 1.2.1 - remove rows with all three information missing (OR Arrival & Departure Time & Total OR Time)
# Identify rows where all three columns are null
rows_to_drop_mask = df['OR Arrival Time'].isnull() & df['OR Departure Time'].isnull() & df['Total Patient OR time'].isnull()
# Drop these rows from the DataFrame
df = df[~rows_to_drop_mask]
print(f"Number of rows dropped: {rows_to_drop_mask.sum()}")

# df[df["Patient"] == 441]

Number of rows dropped: 9


In [ ]:
print(df['Total Patient OR time'].describe())

count    1959.000000
mean       96.938744
std        28.150593
min         0.000000
25%        78.000000
50%        93.000000
75%       112.000000
max       248.000000
Name: Total Patient OR time, dtype: float64


In [ ]:
# 1.2.1 - Abnormal Value
df = df.loc[(df["Total Patient OR time"] > 10) | (df["Total Patient OR time"].isna())]

In [ ]:
# 1.2.3 - Missing Values in “Total Patient OR Time”
print("Missing before:")
print(df["Total Patient OR time"].isna().sum()) # 190 rows
# df[df["Total Patient OR time"].isnull()]

# replaced using the median Total Patient OR Time of that specific CS Type
median_or_time_by_cs_type = df.groupby('CS type')['Total Patient OR time'].median()
df['Total Patient OR time'] = df.apply(lambda row: median_or_time_by_cs_type[row['CS type']] if pd.isnull(row['Total Patient OR time']) else row['Total Patient OR time'], axis=1)
print(f"Missing values after imputation: {df['Total Patient OR time'].isnull().sum()}")

Missing before:
190
Missing values after imputation: 0


### 1.3 - Impute Missing OR Arrival & Departure Time

- Imputation - based on
  - OR Departure Time = OR Arrival Time + Total Patient OR Time
  - OR Arrival Time = OR Departure Time − Total Patient OR Time

<!-- Step 2: Computing Missing “OR Departure Time”
Some records in the dataset had missing values for “OR Departure Time.” Since “OR Departure Time” is determined by when a patient arrives in the OR and how long they stay, missing values were calculated using the formula: OR Departure Time = OR Arrival Time + Total Patient OR Time. The correct departure time was derived using the formula for each row where “OR Arrival Time” was available, but “OR Departure Time” was missing.

Step 3: Computing Missing “OR Arrival Time”
Similar to the previous step, there were missing values in “OR Arrival Time.” If “OR Departure Time” was present but “OR Arrival Time” was missing, it was calculated using the formula: OR Arrival Time = OR Departure Time − Total Patient OR Time. -->

<!-- Convert 'Total Patient OR time' column to a Timedelta object, then impute missing 'OR Departure Time' using the formula 'OR Arrival Time' + 'Total Patient OR Time', and subsequently impute missing 'OR Arrival Time' using 'OR Departure Time' - 'Total Patient OR Time', finally summarizing the number of missing values remaining in 'OR Arrival Time' and 'OR Departure Time'.


To convert the 'Total Patient OR time' column from minutes to a Timedelta object, I will use pd.to_timedelta specifying 'minutes' as the unit.

Now that 'Total Patient OR time' is a Timedelta object, the next step is to impute missing 'OR Departure Time' using the formula 'OR Arrival Time' + 'Total Patient OR Time', as outlined in the overall task description and the previous markdown text. I will first display the count of missing values in 'OR Departure Time' before imputation.

 Following the imputation of 'OR Departure Time', the next step is to impute missing 'OR Arrival Time' using the formula 'OR Departure Time' - 'Total Patient OR Time', as described in the overall task. I will first display the count of missing values in 'OR Arrival Time' before imputation.

Now that both 'OR Departure Time' and 'OR Arrival Time' have been imputed, the final step for this subtask is to summarize the remaining missing values in these columns to confirm the imputation was successful and to verify the overall task's completion.

* The 'Total Patient OR time' column, initially in minutes, was successfully converted to a Timedelta object, appearing in the format 0 days HH:MM:SS.
* Missing values in 'OR Departure Time' were completely imputed. Before imputation, there were 174 missing values, which were reduced to 0 after applying the formula 'OR Arrival Time' + 'Total Patient OR time'.
* Missing values in 'OR Arrival Time' were completely imputed. Initially, there were 13 missing values, which were reduced to 0 after applying the formula 'OR Departure Time' - 'Total Patient OR time'.
* Following the imputation steps, both 'OR Arrival Time' and 'OR Departure Time' columns now have 0 missing values. -->


In [ ]:
# 1 — Convert duration from minutes to a Timedelta
df["Adjusted Total Patient OR time"] = pd.to_timedelta(df["Total Patient OR time"], unit="minutes")

# converted duration format (0 days HH:MM:SS)
print("Converted 'Total Patient OR time' to Timedelta:")
print(df["Adjusted Total Patient OR time"].head())

# 2 — Impute missing OR Departure Time using: Departure = Arrival + Duration
# (Only rows with missing departure are updated; others are left unchanged.)
missing_departure_before = df["OR Departure Time"].isnull().sum()
print(f"Missing 'OR Departure Time' before imputation: {missing_departure_before}")

df.loc[df["OR Departure Time"].isnull(), "OR Departure Time"] = (
    df["OR Arrival Time"] + df["Adjusted Total Patient OR time"]
)

missing_departure_after = df["OR Departure Time"].isnull().sum()
print(f"Missing 'OR Departure Time' after imputation: {missing_departure_after}")

# 3 — Impute missing OR Arrival Time using: Arrival = Departure - Duration
# (Only rows with missing arrival are updated; others are left unchanged.)
missing_arrival_before = df["OR Arrival Time"].isnull().sum()
print(f"Missing 'OR Arrival Time' before imputation: {missing_arrival_before}")

df.loc[df["OR Arrival Time"].isnull(), "OR Arrival Time"] = (
    df["OR Departure Time"] - df["Adjusted Total Patient OR time"]
)

missing_arrival_after = df["OR Arrival Time"].isnull().sum()
print(f"Missing 'OR Arrival Time' after imputation: {missing_arrival_after}")

# confirm there are no remaining missing OR timestamps
print(f"Final missing 'OR Arrival Time': {df['OR Arrival Time'].isnull().sum()}")
print(f"Final missing 'OR Departure Time': {df['OR Departure Time'].isnull().sum()}")

# drop helper column "Adjusted Total Patient OR time"
df = df.drop(columns=['Adjusted Total Patient OR time'])


Converted 'Total Patient OR time' to Timedelta:
0   0 days 03:04:00
2   0 days 01:24:00
3   0 days 01:18:00
4   0 days 01:09:00
5   0 days 01:25:00
Name: Adjusted Total Patient OR time, dtype: timedelta64[ns]
Missing 'OR Departure Time' before imputation: 174
Missing 'OR Departure Time' after imputation: 0
Missing 'OR Arrival Time' before imputation: 13
Missing 'OR Arrival Time' after imputation: 0
Final missing 'OR Arrival Time': 0
Final missing 'OR Departure Time': 0


### 1.4 - Checking 'Planned-as scheduled' OR Arrival Time outside working hours

- to identify any 'Planned-as scheduled' CS types where the 'OR Arrival Time' falls outside of the standard working hours (Monday-Friday, 9 AM to 5 PM). This helps in understanding potential operational discrepancies or scheduled procedures occurring at non-standard times.

In [ ]:
# Step 1 — Subset the dataset to only "Planned-as scheduled" C-section types for targeted validation
planned_surgeries = df[df["CS type"] == "Planned-as scheduled"].copy()

# Step 2 — Extract day-of-week and hour from OR Arrival Time to evaluate whether timing matches the label
# dt.day_name(): returns weekday name (e.g., Monday)
# dt.hour: returns hour in 0–23
planned_surgeries.loc[:, "OR_Arrival_DayOfWeek"] = planned_surgeries["OR Arrival Time"].dt.day_name()
planned_surgeries.loc[:, "OR_Arrival_Hour"] = planned_surgeries["OR Arrival Time"].dt.hour

# Step 3 — Define the expected "working hours" window for scheduled cases (Mon–Fri, 9:00–17:00)
working_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
start_hour = 9
end_hour = 17  # 17:00 means up to but not including 5 PM (i.e., [9, 17))

# Step 4 — Flag "Planned-as scheduled" surgeries that occurred outside the defined window
# Conditions:
#   1) Not on a weekday, OR
#   2) Before 9:00, OR
#   3) At/after 17:00
surgeries_outside_working_hours = planned_surgeries[
    (~planned_surgeries["OR_Arrival_DayOfWeek"].isin(working_days))
    | (planned_surgeries["OR_Arrival_Hour"] < start_hour)
    | (planned_surgeries["OR_Arrival_Hour"] >= end_hour)
].copy()

# Step 5 — Summary output: how many "scheduled" surgeries fall outside the expected time window
count_outside = len(surgeries_outside_working_hours)
print(f"Number of 'Planned-as scheduled' surgeries outside weekdays (9 AM - 5 PM): {count_outside}")

# Step 6 — If any records are flagged, display key columns for manual inspection
# Note: Only include columns that definitely exist; avoid referencing columns that may not be in the raw data.
if count_outside > 0:
    print("Details of flagged 'Planned-as scheduled' surgeries (outside working hours):")
    display_cols = [
        "Patient",
        "Delivery Date",
        "Delivery Time",
        "OR Arrival Time",
        "OR Departure Time",
        "OR_Arrival_DayOfWeek",
        "OR_Arrival_Hour",
        "CS type",
    ]
    # Keep only columns that exist in the dataframe to prevent KeyError
    display_cols = [c for c in display_cols if c in surgeries_outside_working_hours.columns]
    display(surgeries_outside_working_hours[display_cols])
else:
    print("No 'Planned-as scheduled' surgeries were found outside the specified working hours.")


Number of 'Planned-as scheduled' surgeries outside weekdays (9 AM - 5 PM): 196
Details of flagged 'Planned-as scheduled' surgeries (outside working hours):


,Patient,Delivery Date,Delivery Time,OR Arrival Time,OR Departure Time,OR_Arrival_DayOfWeek,OR_Arrival_Hour,CS type
5,6,2013-04-02,08:28:00,2013-04-02 07:53:40.260,2013-04-02 09:19:08.260,Tuesday,7,Planned-as scheduled
13,14,2013-04-03,08:11:00,2013-04-03 07:43:19.713,2013-04-03 08:37:06.713,Wednesday,7,Planned-as scheduled
26,27,2013-04-05,03:25:00,2013-04-05 02:51:59.653,2013-04-05 04:51:04.653,Friday,2,Planned-as scheduled
27,28,2013-04-05,09:31:00,2013-04-05 08:52:53.587,2013-04-05 11:11:58.587,Friday,8,Planned-as scheduled
39,40,2013-04-08,08:09:00,2013-04-08 07:32:36.270,2013-04-08 08:56:52.270,Monday,7,Planned-as scheduled
...,...,...,...,...,...,...,...,...
2163,2164,2014-03-24,08:11:00,2014-03-24 07:44:31.117,2014-03-24 09:02:01.117,Monday,7,Planned-as scheduled
2168,2169,2014-03-25,08:41:00,2014-03-25 07:26:55.420,2014-03-25 09:52:07.420,Tuesday,7,Planned-as scheduled
2175,2176,2014-03-26,08:22:00,2014-03-26 07:54:52.260,2014-03-26 08:57:23.260,Wednesday,7,Planned-as scheduled
2187,2188,2014-03-28,10:03:00,2014-03-28 08:46:30.807,2014-03-28 10:44:29.807,Friday,8,Planned-as scheduled


### 1.5 - check if 'OR Departure Time' is earlier than 'OR Arrival Time' and report the number of such instances.

- found 3 records where the 'OR Departure Time' is earlier than the 'OR Arrival Time'

In [ ]:
earlier_departure_count = (df["OR Departure Time"] < df["OR Arrival Time"]).sum()
print(f"Number of records where 'OR Departure Time' is earlier than 'OR Arrival Time': {earlier_departure_count}")
if earlier_departure_count > 0:
    print("Details of records with 'OR Departure Time' earlier than 'OR Arrival Time':")
    display(df[df['OR Departure Time'] < df['OR Arrival Time']][['Patient', 'Delivery Time','OR Arrival Time', 'OR Departure Time', 'Total Patient OR time']])

# delete
df = df[df["OR Departure Time"] >= df["OR Arrival Time"]]

# check
(df["OR Departure Time"] < df["OR Arrival Time"]).sum()


Number of records where 'OR Departure Time' is earlier than 'OR Arrival Time': 3
Details of records with 'OR Departure Time' earlier than 'OR Arrival Time':


,Patient,Delivery Time,OR Arrival Time,OR Departure Time,Total Patient OR time
644,645,13:58:00,2013-07-15 15:49:46.263,2013-07-15 15:42:06.263,91.0
1000,1001,15:29:00,2013-09-09 19:34:56.530,2013-09-09 16:25:15.530,103.0
1877,1878,10:28:00,2014-02-05 20:01:54.027,2014-02-04 11:32:01.027,94.0


np.int64(0)

### 1.6 - check if the 'Delivery Time' is outside the 'OR Arrival Time' and 'OR Departure Time' range. This will involve combining the delivery date and time into a single datetime column for accurate comparison.

In [ ]:
# Combine 'Delivery Date' and 'Delivery Time' into a single datetime column
# Use errors='coerce' to turn unparseable values (e.g., from 'nan') into NaT
df['Delivery_DateTime'] = pd.to_datetime(
    df['Delivery Date'].astype(str) + ' ' + df['Delivery Time'].astype(str),
    errors='coerce'
)

# Filter out rows where Delivery_DateTime is NaT, as these cannot be meaningfully compared
df_comparable = df.dropna(subset=['Delivery_DateTime']).copy()

# Check if Delivery_DateTime is outside the OR Arrival Time and OR Departure Time range
out_of_range_deliveries = df_comparable[
    (df_comparable['Delivery_DateTime'] < df_comparable['OR Arrival Time']) |
    (df_comparable['Delivery_DateTime'] > df_comparable['OR Departure Time'])
]

print(f"Number of records where 'Delivery Time' is outside 'OR Arrival Time' and 'OR Departure Time' range: {len(out_of_range_deliveries)}")

if not out_of_range_deliveries.empty:
    print("Details of records where 'Delivery Time' is out of range:")
    display(out_of_range_deliveries[['Patient', 'Delivery_DateTime', 'OR Arrival Time', 'OR Departure Time', 'Total Patient OR time', 'CS type']])
else:
    print("No 'Delivery Time' found outside the 'OR Arrival Time' and 'OR Departure Time' range.")

Number of records where 'Delivery Time' is outside 'OR Arrival Time' and 'OR Departure Time' range: 44
Details of records where 'Delivery Time' is out of range:


,Patient,Delivery_DateTime,OR Arrival Time,OR Departure Time,Total Patient OR time,CS type
45,46,2013-04-08 15:32:00,2013-04-08 15:36:52.180,2013-04-08 17:05:30.180,88.0,Planned-as scheduled
84,85,2013-04-16 20:15:00,2013-04-16 18:33:43.653,2013-04-16 20:04:43.653,91.0,Unplanned
93,94,2013-04-18 10:56:00,2013-04-18 08:47:21.217,2013-04-18 10:18:21.217,91.0,Unplanned
132,133,2013-04-24 18:38:00,2013-04-24 18:38:02.937,2013-04-24 21:50:49.937,192.0,Unplanned
177,178,2013-05-02 10:28:00,2013-05-02 08:09:36.340,2013-05-02 09:43:36.340,94.0,Planned-as scheduled
261,262,2013-05-17 08:33:00,2013-05-17 09:11:04.997,2013-05-17 10:47:53.997,96.0,Planned-as scheduled
547,548,2013-06-28 01:03:00,2013-06-28 01:04:44.090,2013-06-28 03:26:03.090,141.0,Unplanned
647,648,2013-07-15 00:03:00,2013-07-15 23:17:39.827,2013-07-16 01:06:47.827,109.0,Unplanned
729,730,2013-07-29 11:52:00,2013-07-29 11:56:23.477,2013-07-29 13:27:23.477,91.0,Unplanned
756,757,2013-08-02 14:20:00,2013-08-02 14:42:35.677,2013-08-02 15:10:04.677,27.0,Unplanned


# Save as Excel file

In [ ]:
output_file_path = 'cleaned_A2.xlsx'
df.to_excel(output_file_path, index=False)
print(f"Cleaned data saved successfully to '{output_file_path}'")

Cleaned data saved successfully to 'cleaned_A2.xlsx'
